In [3]:
!git clone -b atm-10 http://github.com/Zapy67/ATML-PA-4 

fatal: destination path 'ATML-PA-4' already exists and is not an empty directory.


In [49]:
!git pull  http://github.com/Zapy67/ATML-PA-4 atm-10

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 1.60 KiB | 409.00 KiB/s, done.
From http://github.com/Zapy67/ATML-PA-4
 * branch            atm-10     -> FETCH_HEAD
Updating a971c35..99594b8
Fast-forward
 fed_lib/fed_methods.py |   2 +-
 fedsam.ipynb           | 146 ++++++++++---------------------------------------
 2 files changed, 31 insertions(+), 117 deletions(-)


In [4]:
%cd ATML-PA-4

/kaggle/working/ATML-PA-4


In [5]:
!ls

'ATML - PA 4.pdf'   fed_lib	   LICENSE     task1.ipynb   task3.ipynb
 data		    fedsam.ipynb   README.md   task2.ipynb   task4.ipynb


#  FedSAM on heterogeneous domains

In [50]:
import torch
import torch.nn as nn

In [51]:
import fed_lib
import importlib
importlib.reload(fed_lib)


<module 'fed_lib' from '/kaggle/working/ATML-PA-4/fed_lib/__init__.py'>

In [6]:
!pip install fedlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.1 MB/s eta 

In [52]:
from fed_lib import fed_methods, fed_model, utils
importlib.reload(utils)
importlib.reload(fed_methods)
importlib.reload(fed_model)

<module 'fed_lib.fed_model' from '/kaggle/working/ATML-PA-4/fed_lib/fed_model.py'>

In [53]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [54]:
trainset, testset = utils.get_cifar10()

In [11]:
client_loaders = utils.get_heterogenous_domains(trainset, clients=5, balance=False, min_require_size=20, alpha=2, verbose=True)

In [12]:
for idx, loader in enumerate(client_loaders):
    utils.calculate_label_skew(loader.dataset)

Label Distribution (P(Y)): [0.029 0.024 0.045 0.119 0.14  0.154 0.074 0.077 0.109 0.229]
Entropy of Label Distribution H(P(Y)): 2.0241916251853267
Normalized Entropy of Label Distribution: 0.8790952531327627

Label Distribution (P(Y)): [0.022 0.177 0.096 0.097 0.033 0.023 0.023 0.115 0.289 0.126]
Entropy of Label Distribution H(P(Y)): 1.903958168937502
Normalized Entropy of Label Distribution: 0.8268785265441764

Label Distribution (P(Y)): [0.192 0.107 0.152 0.053 0.097 0.128 0.093 0.084 0.095]
Entropy of Label Distribution H(P(Y)): 2.052622759553384
Normalized Entropy of Label Distribution: 0.9341888766062585

Label Distribution (P(Y)): [0.179 0.143 0.182 0.051 0.082 0.118 0.136 0.108]
Entropy of Label Distribution H(P(Y)): 1.9413124560002393
Normalized Entropy of Label Distribution: 0.9335739510291732

Label Distribution (P(Y)): [0.055 0.069 0.027 0.174 0.13  0.061 0.146 0.117 0.066 0.155]
Entropy of Label Distribution H(P(Y)): 2.0874715519155993
Normalized Entropy of Label Distribut

In [13]:
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]
domains = None

In [14]:
client_weights

[0.19882, 0.15042, 0.2084, 0.21338, 0.22898]

In [55]:
partition = "dirichlet"
alpha = 0.1
num_clients = 5
domains=None
fed = fed_model.Federation(num_clients, None, partition, domains, alpha, device, batch_size=64, pin_memory=True, num_workers=2)

In [56]:
client_loaders = fed.client_dataloaders
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]

In [57]:
fed.set_method(fed_methods.FedSAM(sam_rho=0, 
             num_local_steps=len(trainset)/num_clients, 
             client_aggregation_weights= client_weights))

In [58]:
fed.train(2, lr=2000, verbose=False, momentum = 0.01, weight_decay=0.2)


--- Round 1/2 ---
Training Clients
Training Client 1/5


TypeError: FedSAM._train_client() got multiple values for argument 'device'

In [127]:
fed.clients[0]

SmallCNN(
  (backbone): Sequential(
    (0): SmallConvBlock(in_channels=3, out_channels=32)
    (1): SmallConvBlock(in_channels=32, out_channels=64)
    (2): AvgPool2d(kernel_size=1, stride=1, padding=0)
  )
  (task_head): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
  )
)